In [ ]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

# Dataset 
a) Download the first 6 classes (class 0 to 5) of "German Traffic Sign Recognition Benchmark" dataset from Kaggle
https://www.kaggle.com/datasets/aakcodebreaker/german-traffic-sign-recognition-benchmark . Copy both training and test datasets.

b) Calculate the mean and standard deviation of the training images.

In [ ]:
# write your code here


mean = None
std_dev = None

c) Implement the custom dataset class for training and test.  Don't use Pytorch ``ImageFolder``. 
For training, do the following transforms and data augmentation: 
 - resize the images to 64 by 64.
 - Horizontal flip with a probabilty of 0.25
 - Vertical flip with a probabilty of 0.25
 - random rotation between 0 and 10 degrees.
 - Convert ``dtype`` to ``float32`` and scale between $0$ and $1$
 - then Normalize with the mean and std. dev. obtained in step b.
 
For testing, resize to 64 by 64, convert ``dtype`` to ``float32`` and scale between $0$ and $1$ and normalize.

(i) In the init method, store the path to the images and labels. You can use any data structure of your choice, e.g., list/nested list, dictionary, or pandas dataframe.

(ii) in the getitem method, use the path stored at position ``idx`` to load the image. Return image and its label. 

In [ ]:
class HousingDataset(Dataset):
    def __init__(self, root_dir, training = True, transform=None): # Here root_dir is the path to train or test folder.
        # coomplete this method as per instructions
        
    def __len__(self):
        length = None
        return length
    
    
    def __getitem__(self, idx):
        #here load image and do the transformatoin
        image = None
        label_price = None
        return image, label_price
    

d) Instantiate training and test datasets.

In [ ]:
# code

e) Partition the training dataset further into training and validation. 20% of training images are validation. Configure the dataloaders with batch size of 16.

In [ ]:
# code

In [ ]:

train_loader = None
val_loader = None
test_loader = None

e) Visualize two random training images.

In [ ]:
# your code

# Q1. CNN
a) Build a NN network specified in the figure below and initialize the parameters. 

The network consists of:
1. Convolution layer: 16 3x3 filters with stride=1. Set the padding such that the output size is half of the input. Convolution is followed by BatchNorm and ReLu Activation fuction.
1. Maxpool with 2x2 filters and stride = 2
2. Convolution layer: 16 3x3 filters with stride=2. Set the padding such that the output size is half of the input. Convolution is followed by BatchNorm and ReLu Activation fuction.
1. Convolution layer: 32 3x3 filters. Convolution is followed by BatchNorm and ReLu Activation fuction.
2. Convolution layer: 64 3x3 filters with stride=2. Set the padding such that the output size is half of the input. Convolution is followed by BatchNorm and ReLu Activation fuction.
3. A fully connected output layer.


Note: select padding of the convolution input such that output height and width are an integer division of input size. 


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        # Define the layers/parameters here
        
        
        
        
    def forward(self, input):
        # implement the forward propagation
        
        
        return image, label

b) Create a model

In [ ]:
cnn_model = CNN()

# if you use GPU, uncomment the floolowing line
# cnn_model = cnn_model.cuda()

c) Do forward propagation for a minibatch and verify that the output shape equals the number of samples in a minibatch by the number of classes.

In [ ]:
data,labels = next(iter(train_loader))

output = cnn_model(data)

print(output.shape)

d) Set up the loss and optimizer. 8$. 

In [ ]:
loss_fn = None

# Learning rate
lr = None
epochs = None
# Modify this op
optimizer = torch.optim.SGD(None, None, None, None)

## Q2. Hyperparamter tuning

a) Do grid search to find a good combination of the optimizer hyperparamters. Use 
1. lr=1e-3, 1e-2, 1e-1, 1e-0, 10
1. momentum = 0.85, 0.9, 0.95, 0.99
2. weight decay = 0, 1e-4, 1e-2

You need to use nested loop. Run each experiment for two epochs. 


In [ ]:
# code

b) Plot the training loss. Label each line with corresponding parameters (lr, momentum, and weight decay).  Pick the set of parameters that yields the best loss. 

In [ ]:
# code

## Q3. Training and Evaluation

a) Use the hyperparamters selected in the previous stage to train the model for 50 epochs. Track validation accuracy and save the model whenever validation accuracy is higher. Follow the template and instructions below.

In [ ]:
training_avg_loss = []
val_avg_loss = []
best_val_acc = 0.0
train_acc = []
val_acc = []

for epoch in range(epochs):
    # Training
    
    # Setting the model to training mode
    cnn_model.train()
    
    for batch in iter(train_loader):
        # Setting the gradients to zero. Otherwise, the gradients will be accumulated over iterations
        cnn_model.zero_grad()
        # unpack batch to get data and associate labels
        data, target_labels = batch
        # if you are using GPU, uncomment the following line
#         data, target_labels = data.cuda(), target_labels.cuda()
        
        # forward propagte
        prediction = cnn_model(data)
        # calculating loss
        loss = loss_fn(prediction, target_labels)
        
        
        # back propagation from loss
        loss.backward()
        
        # update the parameters
        optimizer.step()
        
        # write code to calculate average training loss

        # write code to calculate training accuracy
        
        ###### Evaluation ####
        
        # Note, evaluation here is done after each epoch. 
        
    # During validatioin, we do not optimize so no need to build computation graph for gradient. 
    # Hence, turning off gradients using torch.no_grad()
    with torch.no_grad():
        cnn_model.eval()
        for batch in iter(val_loader):
            # unpack batch to get data and associate labels
            data, target_labels = batch
            # forward propagte
            prediction = cnn_model(data)
            # calculating loss
            loss = loss_fn(prediction, target_labels)

        # write code to calculate average validation loss

        # write code to calculate validation accuracy
        

b) Plot losses vs epoch for training and validation datasets

In [ ]:
# your code

c) Load the saved best model.

In [ ]:
# your code

d) Use the loaded model to calculate the accuracy for the test data. Show the results.

In [ ]:
# your code

### Part 2

## Q1. Transfer Learning

a) Use transfer learning to finetune a pretrained ``ResNet18`` model. 

1. Load the pretrained model
2. Get the transform for training and validation.
3. Define the dataset for training, validation, and test again with the new transform.
4. Define dataloader

In [ ]:
# your code

b) 
1. Set the ``requires_grad`` to ``False`` for all parameters of the network.
2. Replace the last layer to adpat to your dataset

In [ ]:
# your code

c) Train and evaluate with a suitable set of hyperparameters and save the best model.

In [ ]:
# code

d) evaluate on the test data and report the results.

In [ ]:
# code